# DFT & eDMFT Calculations

flow: check dependencies -> import libs -> set config -> load dataset -> generate QE input files (SCF & NSCF) -> run DFT -> generate Wannier90 input -> run Wannier90 -> parse tight-binding hamiltonian -> run full eDMFT with TRIQS_CTHYB

## 1. Dependencies and imports

In [1]:
import importlib, shutil, os
import pandas as pd
from IPython.display import display

check_data = []

for pkg in ["numpy", "ase", "jarvis", "triqs", "nglview", "seaborn", "pandas"]:
    try:
        mod = importlib.import_module(pkg)
        version = getattr(mod, '__version__', 'N/A')
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': 'OK', 'Details': version})
    except ImportError:
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': 'MISSING', 'Details': 'Not installed'})

try:
    from triqs.gf import GfImFreq, inverse, iOmega_n, SemiCircular
    from triqs_cthyb import Solver
    from triqs.operators import n
    
    S_test = Solver(beta=10.0, gf_struct=[('up', 1), ('down', 1)])
    S_test.G_iw << SemiCircular(1.0)
    H_test = 2.0 * n('up', 0) * n('down', 0)
    
    for name, g0 in S_test.G0_iw:
        g0 << inverse(iOmega_n + 0.5 - 0.25 * S_test.G_iw[name])
    
    S_test.solve(h_int=H_test, n_cycles=100, length_cycle=10, n_warmup_cycles=50)
    density = S_test.G_iw['up'].density()[0, 0].real
    
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': 'OK', 'Details': f'test density={density:.3f}'})
except Exception as e:
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': 'FAILED', 'Details': str(e)[:50]})

for exe in ["pw.x", "wannier90.x", "pw2wannier90.x", "mpirun"]:
    found = shutil.which(exe)
    status = 'OK' if found else 'MISSING'
    details = found if found else 'Not in PATH'
    check_data.append({'Category': 'Executable', 'Item': exe, 'Status': status, 'Details': details})

pseudo_dir = "/usr/share/espresso/pseudo"
if os.path.isdir(pseudo_dir):
    num_pseudos = len([f for f in os.listdir(pseudo_dir) if f.endswith(('.upf', '.UPF'))])
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': 'OK', 'Details': pseudo_dir})
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'File count', 'Status': 'OK', 'Details': f'{num_pseudos} files'})
else:
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': 'MISSING', 'Details': 'Not found'})

display(pd.DataFrame(check_data))


╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-0.5*c_dag('down',0)*c('down',0) + -0.5*c_dag('up',0)*c('up',0) + 2*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0)
Using autopartition algorithm to partition the local Hilbert space
Found 4 subspaces.

Warming up ...
02:27:45 100% ETA 00:00:00 cycle 49 of 50



Accumulating ...
02:27:45 100% ETA 00:00:00 cycle 99 of 100


[Rank 0] Collect results: Waiting for all mpi-threads to finish accumulating...
[Rank 0] Timings for all measures:
Measure               | seconds   
Auto-correlation time | 1.063e-05 
Average order         | 2.163e-06 
Average sign          | 1.962e-06 
G_tau measure         | 0.000256261
Total measure time    | 0.000271016
[Rank 0] Acceptance rate for all moves:
Move set Insert two operators: 0.13617
  Move  Insert Delta_up: 0.132812
  Move  Insert Delta_down: 0.140187
Move set Remove two operators: 0.188889
  Move  Remove Delt

Starting serial run at: 2026-02-04 02:27:44.979286


,Category,Item,Status,Details
0,Package,numpy,OK,1.26.4
1,Package,ase,OK,3.27.0
2,Package,jarvis,OK,2026.1.10
3,Package,triqs,OK,N/A
4,Package,nglview,OK,4.0
5,Package,seaborn,OK,0.13.2
6,Package,pandas,OK,3.0.0
7,Package,TRIQS CTHYB Runtime,OK,test density=1.100
8,Executable,pw.x,OK,/home/vm/miniconda3/envs/DSI/bin/pw.x
9,Executable,wannier90.x,OK,/home/vm/miniconda3/envs/DSI/bin/wannier90.x


In [2]:
import os
import sys

# ensure conda libs are in ld_library_path for subprocess calls
conda_prefix = os.environ.get('CONDA_PREFIX', os.path.expanduser('~/miniconda3/envs/DSI'))
lib_path = os.path.join(conda_prefix, 'lib')
os.environ['LD_LIBRARY_PATH'] = lib_path + ':' + os.environ.get('LD_LIBRARY_PATH', '')

import numpy as np
import pandas as pd
from IPython.display import display, HTML
from jarvis.db.figshare import data as jarvis_data
from jarvis.core.atoms import Atoms as JarvisAtoms
from ase import Atoms as AseAtoms
from triqs.gf import GfImFreq, BlockGf
import nglview as nv
import matplotlib.pyplot as plt
import seaborn as sns
import re
import subprocess
import multiprocessing

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

## 2. Config

In [3]:
JARVIS_DATABASE = "dft_3d"
WORK_DIR = os.path.join(os.getcwd(), "calculations")
os.makedirs(WORK_DIR, exist_ok=True)

QE_PSEUDOPOTENTIALS_DIR = pseudo_dir
QE_EXECUTABLE = shutil.which("pw.x") or shutil.which("pw")
WANNIER90_EXECUTABLE = shutil.which("wannier90.x") or shutil.which("wannier90")
PW2WANNIER90_EXECUTABLE = shutil.which("pw2wannier90.x") or shutil.which("pw2wannier90")
MPI_EXECUTABLE = shutil.which("mpirun") or shutil.which("mpiexec")

# memory constraints - adjust SYSTEM_RAM_GB for your machine
SYSTEM_RAM_GB = 16
MAX_ATOMS = {8: 2, 16: 6, 32: 12, 64: 25, 128: 50}.get(SYSTEM_RAM_GB, int(SYSTEM_RAM_GB * 0.4))

OCCUPATIONS = "smearing"
SMEARING = "cold"
DEGAUSS = 0.01
DMFT_ITERATIONS = 10

print(f"system ram: {SYSTEM_RAM_GB} GB, max atoms: {MAX_ATOMS}")
print(f"dmft iterations: {DMFT_ITERATIONS}")
print(f"executables: pw.x={QE_EXECUTABLE is not None}, wannier90={WANNIER90_EXECUTABLE is not None}")
print(f"pseudopotentials: {QE_PSEUDOPOTENTIALS_DIR}")
print("\n(nprocs, cutoffs, k-points computed after material is loaded)")

system ram: 16 GB, max atoms: 6
dmft iterations: 10
executables: pw.x=True, wannier90=True
pseudopotentials: /usr/share/espresso/pseudo

(nprocs, cutoffs, k-points computed after material is loaded)


## 3. Load Material from JARVIS

In [4]:
data = jarvis_data(JARVIS_DATABASE)
candidates = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "DFT_eDMFT/candidate_materials.csv"))

MATERIAL_INDEX = 0  # change to process different materials
formula = candidates.iloc[MATERIAL_INDEX]['formula']
mat = next((m for m in data if m.get('formula') == formula), None)
MATERIAL_ID = mat['jid']

print(f"{MATERIAL_INDEX}: {formula} -> {MATERIAL_ID}")

jarvis_atoms = JarvisAtoms.from_dict(mat['atoms'])
ase_atoms = AseAtoms(
    symbols=jarvis_atoms.elements,
    positions=jarvis_atoms.cart_coords,
    cell=jarvis_atoms.lattice_mat,
    pbc=True
)

# check if material exceeds memory limits
n_atoms = len(ase_atoms)
if n_atoms > MAX_ATOMS:
    raise MemoryError(
        f"material has {n_atoms} atoms, max supported on {SYSTEM_RAM_GB}GB RAM is {MAX_ATOMS}.\n"
        f"try a smaller material or run on a machine with more memory."
    )

# MPI processes for Quantum ESPRESSO
# QE distributes memory across processes, so MORE processes = LESS ram per process
# small systems: limited by parallelization efficiency (overhead dominates)
# large systems: need MORE processes to distribute memory, not fewer!
available_cores = multiprocessing.cpu_count()
ram_per_core = SYSTEM_RAM_GB / available_cores

if n_atoms <= 2:
    # small systems: parallel efficiency drops with too many cores, but memory is fine
    # use up to half the cores to balance speed vs overhead
    QE_NPROCS = min(available_cores // 2, 4) or 1
elif n_atoms <= 6:
    # medium systems: can use most cores effectively
    QE_NPROCS = min(available_cores - 1, 8)
else:
    # large systems: NEED more processes to distribute memory load
    # each process gets a fraction of the FFT grid
    QE_NPROCS = available_cores

# ensure at least 2GB per process for QE overhead
min_ram_per_proc = 2.0
max_procs_for_ram = max(1, int(SYSTEM_RAM_GB / min_ram_per_proc))
QE_NPROCS = min(QE_NPROCS, max_procs_for_ram)

print(f"atoms: {n_atoms} (max {MAX_ATOMS} for {SYSTEM_RAM_GB}GB RAM)")
print(f"mpi processes: {QE_NPROCS} (of {available_cores} cores, ~{SYSTEM_RAM_GB/QE_NPROCS:.1f}GB per process)")

Obtaining 3D dataset 76k ...
Reference:https://doi.org/10.1016/j.commatsci.2025.114063
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
0: InP -> JVASP-1183
atoms: 2 (max 6 for 16GB RAM)
mpi processes: 4 (of 10 cores, ~4.0GB per process)


In [5]:
elements = list(set(ase_atoms.get_chemical_symbols()))
element_counts = {e: ase_atoms.get_chemical_symbols().count(e) for e in elements}

mat_props = [
    {'Property': 'Material ID', 'Value': MATERIAL_ID},
    {'Property': 'Formula', 'Value': ase_atoms.get_chemical_formula()},
    {'Property': 'Atoms', 'Value': len(ase_atoms)},
    {'Property': 'Elements', 'Value': str(element_counts)},
    {'Property': 'Volume', 'Value': f'{ase_atoms.get_volume():.3f} A^3'},
    {'Property': 'Cell', 'Value': f'{ase_atoms.cell.lengths()[0]:.3f} x {ase_atoms.cell.lengths()[1]:.3f} x {ase_atoms.cell.lengths()[2]:.3f} A'},
]

for key in ['formation_energy_peratom', 'optb88vdw_bandgap', 'spillage']:
    if key in mat and mat[key] is not None:
        val = mat[key]
        mat_props.append({'Property': key, 'Value': f'{val:.4f}' if isinstance(val, (int, float)) else str(val)})

display(pd.DataFrame(mat_props))

,Property,Value
0,Material ID,JVASP-1183
1,Formula,InP
2,Atoms,2
3,Elements,"{'In': 1, 'P': 1}"
4,Volume,52.799 A^3
5,Cell,4.211 x 4.211 x 4.211 A
6,formation_energy_peratom,-0.2243
7,optb88vdw_bandgap,0.3310
8,spillage,na


In [6]:
from ase.build import make_supercell

supercell = make_supercell(ase_atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
view = nv.show_ase(supercell)
view.add_unitcell()
view.center()
view

NGLWidget()

## 4. Generate Quantum ESPRESSO Input Files (SCF & NSCF)

In [7]:
# parse upf pseudopotential header to extract valence, angular momentum, type
def parse_upf_header(upf_path):
    info = {'z_valence': None, 'l_max': None, 'pseudo_type': None, 'element': None}
    with open(upf_path, 'r') as f:
        content = f.read()
    header_match = re.search(r'<PP_HEADER([^>]+)>', content, re.DOTALL)
    if header_match:
        header = header_match.group(1)
        for key in ['z_valence', 'l_max', 'pseudo_type', 'element']:
            match = re.search(rf'{key}="([^"]+)"', header)
            if match:
                val = match.group(1).strip()
                info[key] = float(val) if key in ['z_valence', 'l_max'] else val
    return info

# estimate cutoffs from element properties: heavier elements and higher l_max need more
def get_recommended_cutoffs(element, z_valence, l_max, pseudo_type):
    from ase.data import atomic_numbers
    Z = atomic_numbers.get(element, 30)
    row = 1 + (Z > 2) + (Z > 10) + (Z > 18) + (Z > 36) + (Z > 54) + (Z > 86)
    
    base_wfc = 30.0
    if l_max is not None:
        base_wfc += l_max * 8.0  # d/f orbitals need higher cutoffs
    if z_valence is not None:
        base_wfc += z_valence * 1.5  # more electrons = higher cutoff
    base_wfc += (row - 1) * 3.0
    
    # paw needs higher rho multiplier, but cap for memory
    rho_multiplier = 8.0 if pseudo_type == 'PAW' else 4.0
    ecutwfc = min(50.0, max(35.0, round(base_wfc / 5) * 5))  # cap at 50 Ry for RAM
    ecutrho = min(400.0, ecutwfc * rho_multiplier)  # cap at 400 Ry
    return ecutwfc, ecutrho

# classify elements by dominant orbital character for wannier projections
def get_element_orbital_type(element):
    d_metals = {'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn',
                'Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd',
                'Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg'}
    f_metals = {'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu',
                'Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf'}
    s_metals = {'Li','Na','K','Rb','Cs','Fr'}
    sp_metals = {'Be','Mg','Ca','Sr','Ba','Ra','Al','Ga','In','Tl'}
    
    if element in d_metals:
        return 'd', 5
    elif element in f_metals:
        return 'f', 7
    elif element in s_metals:
        return 's', 1
    elif element in sp_metals:
        return 'sp', 2
    else:
        return 'sp3', 4  # main group defaults to sp3 hybridized

# estimate hubbard U from element type: 3d > 4d > 5d, 4f > 5f
def get_hubbard_u(element):
    u_values = {'d_3d': 4.0, 'd_4d': 3.0, 'd_5d': 2.0, 'f_4f': 6.0, 'f_5f': 4.0}
    d_3d = {'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn'}
    d_4d = {'Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd'}
    d_5d = {'Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg'}
    f_4f = {'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu'}
    f_5f = {'Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf'}
    
    if element in d_3d: return u_values['d_3d']
    if element in d_4d: return u_values['d_4d']
    if element in d_5d: return u_values['d_5d']
    if element in f_4f: return u_values['f_4f']
    if element in f_5f: return u_values['f_5f']
    return 0.0  # non-correlated

# calculate k-point grid from cell size (smaller cell = more k-points)
# caps total k-points for 16GB RAM - 500 k-pts is sufficient for most materials
def get_kpoints_from_cell(cell, density=0.10, max_total_kpts=500):
    recip = 2 * np.pi * np.linalg.inv(cell).T
    recip_lengths = np.linalg.norm(recip, axis=1)
    kpts = [max(1, int(np.ceil(length / density))) for length in recip_lengths]
    
    # scale down proportionally if exceeding max
    total = kpts[0] * kpts[1] * kpts[2]
    if total > max_total_kpts:
        scale = (max_total_kpts / total) ** (1/3)
        kpts = [max(1, int(k * scale)) for k in kpts]
    return kpts

# collect pseudopotentials and compute all dynamic parameters
elements = list(set(ase_atoms.get_chemical_symbols()))
pseudos = {}
pp_info = {}
cutoffs_wfc = []
cutoffs_rho = []

for e in elements:
    cands = [f for f in os.listdir(QE_PSEUDOPOTENTIALS_DIR) 
             if f.lower().endswith('.upf') and f.startswith(e + '.')]
    if not cands:
        raise FileNotFoundError(f"no pseudopotential for '{e}' in {QE_PSEUDOPOTENTIALS_DIR}")
    # prefer non-semicore version for standard calculations
    non_sp = [c for c in cands if '_sp' not in c]
    pp_file = non_sp[0] if non_sp else cands[0]
    pseudos[e] = pp_file
    
    pp_path = os.path.join(QE_PSEUDOPOTENTIALS_DIR, pp_file)
    info = parse_upf_header(pp_path)
    pp_info[e] = info
    
    wfc, rho = get_recommended_cutoffs(e, info['z_valence'], info['l_max'], info['pseudo_type'])
    cutoffs_wfc.append(wfc)
    cutoffs_rho.append(rho)

# use max cutoffs across all elements (qe requires consistent values)
ECUTWFC = max(cutoffs_wfc)
ECUTRHO = max(cutoffs_rho)

# compute wannier function count and identify correlated elements
num_wann = 0
orbital_info = {}
correlated_elements = []
for e in elements:
    n_atoms = ase_atoms.get_chemical_symbols().count(e)
    orb_type, orb_count = get_element_orbital_type(e)
    orbital_info[e] = {'type': orb_type, 'count': orb_count, 'atoms': n_atoms}
    num_wann += orb_count * n_atoms
    if orb_type in ['d', 'f']:
        correlated_elements.append(e)

NUM_WANNIER_FUNCTIONS = num_wann
K_POINTS = get_kpoints_from_cell(ase_atoms.cell)

u_values = [get_hubbard_u(e) for e in correlated_elements]
HUBBARD_U = max(u_values) if u_values else 0.0

print("pseudopotentials and parameters:")
for e in elements:
    info = pp_info[e]
    wfc, rho = get_recommended_cutoffs(e, info['z_valence'], info['l_max'], info['pseudo_type'])
    orb = orbital_info[e]
    u = get_hubbard_u(e)
    print(f"  {e}: {pseudos[e]}, z={info['z_valence']}, l_max={int(info['l_max']) if info['l_max'] else '?'}, "
          f"cutoff={wfc}/{rho} Ry, orbitals={orb['type']}x{orb['atoms']}, U={u} eV")

print(f"\ncutoffs: {ECUTWFC}/{ECUTRHO} Ry")
print(f"k-points (scf): {K_POINTS} = {K_POINTS[0]*K_POINTS[1]*K_POINTS[2]} pts")
print(f"wannier functions: {NUM_WANNIER_FUNCTIONS}")
print(f"correlated elements: {correlated_elements}, U={HUBBARD_U} eV")

# nscf k-grid: same as scf (doubling causes OOM for large systems)
# wannier90 interpolates anyway, so dense scf grid is sufficient
nk_nscf = K_POINTS
nkpts_nscf = nk_nscf[0] * nk_nscf[1] * nk_nscf[2]
print(f"k-points (nscf): {nk_nscf} = {nkpts_nscf} pts")
kpoints_nscf = [[i/nk_nscf[0], j/nk_nscf[1], k/nk_nscf[2]] 
                for k in range(nk_nscf[2]) 
                for j in range(nk_nscf[1]) 
                for i in range(nk_nscf[0])]

def generate_qe_input(calc_type, k_grid, atoms, material_id, pseudos, nbnd=None):
    lines = [
        f"&CONTROL\n  calculation='{calc_type}'\n  prefix='{material_id}'\n",
        f"  outdir='./tmp'\n  pseudo_dir='{QE_PSEUDOPOTENTIALS_DIR}'\n/\n",
        f"&SYSTEM\n  ibrav=0\n  nat={len(atoms)}\n  ntyp={len(elements)}\n",
        f"  ecutwfc={ECUTWFC}\n  ecutrho={ECUTRHO}\n",
        f"  occupations='{OCCUPATIONS}'\n  smearing='{SMEARING}'\n  degauss={DEGAUSS}\n",
    ]
    if calc_type == 'nscf':
        lines.append("  nosym=.true.\n  noinv=.true.\n")
        if nbnd is not None:
            lines.append(f"  nbnd={nbnd}\n")
    lines.append("/\n&ELECTRONS\n  conv_thr=1.0d-8\n")
    if calc_type == 'nscf':
        # use conjugate gradient to avoid s-matrix issues with paw
        lines.append("  diagonalization='cg'\n  diago_full_acc=.true.\n")
    lines.append("/\nATOMIC_SPECIES\n")
    for e in elements:
        mass = atoms.get_masses()[atoms.get_chemical_symbols().index(e)]
        lines.append(f"  {e}  {mass:.4f}  {pseudos[e]}\n")
    lines.append("\nCELL_PARAMETERS angstrom\n")
    for i in range(3):
        lines.append(f"  {atoms.cell[i,0]:16.10f} {atoms.cell[i,1]:16.10f} {atoms.cell[i,2]:16.10f}\n")
    lines.append("\nATOMIC_POSITIONS angstrom\n")
    for s, p in zip(atoms.get_chemical_symbols(), atoms.positions):
        lines.append(f"  {s:4s} {p[0]:16.10f} {p[1]:16.10f} {p[2]:16.10f}\n")
    if calc_type == 'scf':
        lines.append(f"\nK_POINTS automatic\n  {k_grid[0]} {k_grid[1]} {k_grid[2]}  0 0 0\n")
    else:
        lines.append(f"\nK_POINTS crystal\n  {nkpts_nscf}\n")
        for kpt in kpoints_nscf:
            lines.append(f"  {kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}  1.0\n")
    return ''.join(lines)

qe_scf_input = generate_qe_input('scf', K_POINTS, ase_atoms, MATERIAL_ID, pseudos)
qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos)

pseudopotentials and parameters:
  In: In.GGA_PBE-JTH.UPF, z=13.0, l_max=2, cutoff=50.0/400.0 Ry, orbitals=spx1, U=0.0 eV
  P: P.GGA_PBE-JTH.UPF, z=5.0, l_max=1, cutoff=50.0/400.0 Ry, orbitals=sp3x1, U=0.0 eV

cutoffs: 50.0/400.0 Ry
k-points (scf): [7, 7, 7] = 343 pts
wannier functions: 6
correlated elements: [], U=0.0 eV
k-points (nscf): [7, 7, 7] = 343 pts


## 5. Run DFT Calculations

In [8]:
def run_command(cmd, cwd, timeout=None):
    process = subprocess.Popen(
        cmd, shell=True, cwd=cwd,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    output_lines = []
    try:
        for line in process.stdout:
            print(line, end='', flush=True)
            output_lines.append(line)
        process.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        process.kill()
        raise
    if process.returncode != 0:
        raise RuntimeError(f"command failed with code {process.returncode}")
    return ''.join(output_lines)

dft_dir = os.path.join(WORK_DIR, f"dft_{MATERIAL_ID}")
os.makedirs(dft_dir, exist_ok=True)

# clean tmp to avoid stale wavefunction data from previous runs
tmp_dir = os.path.join(dft_dir, "tmp")
if os.path.exists(tmp_dir):
    shutil.rmtree(tmp_dir)
    print(f"cleaned {tmp_dir}")

with open(os.path.join(dft_dir, f"{MATERIAL_ID}_scf.in"), 'w') as f:
    f.write(qe_scf_input)

print("running scf...")
scf_output = run_command(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_scf.in",
    cwd=dft_dir
)

# determine nscf bands: enough for wannier but capped to avoid s-matrix issues
nbnd_match = re.search(r'number of Kohn-Sham states\s*=\s*(\d+)', scf_output)
scf_nbnd = int(nbnd_match.group(1)) if nbnd_match else NUM_WANNIER_FUNCTIONS
nscf_nbnd = max(scf_nbnd * 2, int(NUM_WANNIER_FUNCTIONS * 1.5))
nscf_nbnd = min(nscf_nbnd, scf_nbnd * 3)
print(f"nscf will use {nscf_nbnd} bands (scf had {scf_nbnd})")

qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos, nbnd=nscf_nbnd)
with open(os.path.join(dft_dir, f"{MATERIAL_ID}_nscf.in"), 'w') as f:
    f.write(qe_nscf_input)

print("running nscf...")
run_command(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_nscf.in",
    cwd=dft_dir
)

cleaned /home/vm/LUMENS-PV/DFT_eDMFT/calculations/dft_JVASP-1183/tmp
running scf...

     Program PWSCF v.7.5 starts on  4Feb2026 at  2:27:49 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       4 processor cores
     Number of MPI processes:                 4
     Threads/MPI process:                     1

     MPI processes distributed on     1 nodes
     844 MiB available memory on the printing compute node when the environment starts

     Reading input from JVASP-1183_scf.in

   

'\n     Program PWSCF v.7.5 starts on  4Feb2026 at  2:27:57 \n\n     This program is part of the open-source Quantum ESPRESSO suite\n     for quantum simulation of materials; please cite\n         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);\n         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);\n         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);\n          URL http://www.quantum-espresso.org", \n     in publications or presentations arising from this work. More details at\n     http://www.quantum-espresso.org/quote\n\n     Parallel version (MPI & OpenMP), running on       4 processor cores\n     Number of MPI processes:                 4\n     Threads/MPI process:                     1\n\n     MPI processes distributed on     1 nodes\n     812 MiB available memory on the printing compute node when the environment starts\n\n     Reading input from JVASP-1183_nscf.in\n\n     Current dimensions of program PWSCF are:\n     Max number o

## 6. Generate Wannier90 Input

In [9]:
seedname = "wan"
nk = nk_nscf  # use same grid as nscf
kpoints = [[i/nk[0], j/nk[1], k/nk[2]] 
           for k in range(nk[2]) for j in range(nk[1]) for i in range(nk[0])]

projections = [f"{e}:{orbital_info[e]['type']}" for e in elements]
proj_str = '\n'.join(projections)
print(f"projections: {projections}")

wannier_lines = [
    f"num_wann = {NUM_WANNIER_FUNCTIONS}\n",
    f"num_bands = {nscf_nbnd}\n",
    "num_iter = 200\n",
    "dis_num_iter = 200\n",
    "write_hr = true\n",
    f"begin projections\n{proj_str}\nend projections\n",
    "begin unit_cell_cart\nang\n",
]
for i in range(3):
    wannier_lines.append(f"{ase_atoms.cell[i,0]:16.10f} {ase_atoms.cell[i,1]:16.10f} {ase_atoms.cell[i,2]:16.10f}\n")
wannier_lines.append("end unit_cell_cart\n")
wannier_lines.append("begin atoms_frac\n")
for s, p in zip(ase_atoms.get_chemical_symbols(), ase_atoms.get_scaled_positions()):
    wannier_lines.append(f"{s} {p[0]:.10f} {p[1]:.10f} {p[2]:.10f}\n")
wannier_lines.append("end atoms_frac\n")
wannier_lines.append(f"mp_grid = {nk[0]} {nk[1]} {nk[2]}\n")
wannier_lines.append("begin kpoints\n")
for kpt in kpoints:
    wannier_lines.append(f"{kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}\n")
wannier_lines.append("end kpoints\n")

wannier_input = ''.join(wannier_lines)
print(f"wannier90: num_wann={NUM_WANNIER_FUNCTIONS}, num_bands={nscf_nbnd}")

projections: ['In:sp', 'P:sp3']
wannier90: num_wann=6, num_bands=26


## 7. Run Wannier90 Workflow

In [10]:
wan_dir = os.path.join(WORK_DIR, f"wannier_{MATERIAL_ID}")
os.makedirs(wan_dir, exist_ok=True)

with open(os.path.join(wan_dir, f"{seedname}.win"), 'w') as f:
    f.write(wannier_input)

print("wannier90 preprocessing...")
run_command(f"{WANNIER90_EXECUTABLE} -pp {seedname}", cwd=wan_dir)

pw2wan_input = f"""&inputpp
  outdir = '{dft_dir}/tmp'
  prefix = '{MATERIAL_ID}'
  seedname = '{seedname}'
  write_mmn = .true.
  write_amn = .true.
  write_unk = .false.
/
"""
with open(os.path.join(wan_dir, "pw2wan.in"), 'w') as f:
    f.write(pw2wan_input)

print("pw2wannier90...")
run_command(f"{PW2WANNIER90_EXECUTABLE} < pw2wan.in", cwd=wan_dir)

print("wannier90...")
run_command(f"{WANNIER90_EXECUTABLE} {seedname}", cwd=wan_dir)

hr_file = os.path.join(wan_dir, f"{seedname}_hr.dat")
with open(hr_file, 'r') as f:
    hr_data = f.read()
print(f"generated {seedname}_hr.dat")

wannier90 preprocessing...
pw2wannier90...

     Program PW2WANNIER v.7.5 starts on  4Feb2026 at  2:29:25 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on      10 processor cores
     Number of MPI processes:                 1
     Threads/MPI process:                    10

     MPI processes distributed on     1 nodes
     394 MiB available memory on the printing compute node when the environment starts


     Reading nscf_save data

     Reading xml data from directory:

     /home/vm

## 8. Parse Tight-Binding Hamiltonian

In [11]:
# parse wannier90 hr.dat: hopping matrix H(R) for all R vectors
lines = hr_data.strip().split('\n')
num_wann = int(lines[1])
nrpts = int(lines[2])
ndegen_lines = (nrpts + 14) // 15

# degeneracy weights for each R vector
degen = []
for i in range(ndegen_lines):
    degen.extend(map(int, lines[3+i].split()))

hopping = {}
for line in lines[3+ndegen_lines:]:
    parts = line.split()
    if len(parts) < 7:
        continue
    R = (int(parts[0]), int(parts[1]), int(parts[2]))
    i, j = int(parts[3])-1, int(parts[4])-1
    t = float(parts[5]) + 1j*float(parts[6])
    if R not in hopping:
        hopping[R] = np.zeros((num_wann, num_wann), dtype=complex)
    hopping[R][i, j] = t

R_list = list(hopping.keys())
R_degen = {R: degen[i] for i, R in enumerate(R_list)}

print(f"wannier hamiltonian: {num_wann} orbitals, {len(R_list)} R-vectors")

# H(k) = sum_R H(R) * exp(i k.R) / degen(R)
def get_Hk(k):
    Hk = np.zeros((num_wann, num_wann), dtype=complex)
    for R, HR in hopping.items():
        phase = np.exp(2j * np.pi * np.dot(k, R))
        Hk += HR * phase / R_degen[R]
    return 0.5 * (Hk + Hk.conj().T)  # enforce hermiticity

# compute bandwidth to set dmft parameters
nk_test = 4
kpts_test = [[i/nk_test, j/nk_test, k/nk_test] 
             for i in range(nk_test) for j in range(nk_test) for k in range(nk_test)]
eigs = [np.linalg.eigvalsh(get_Hk(k)) for k in kpts_test]
all_eigs = np.array(eigs).flatten()
bandwidth = all_eigs.max() - all_eigs.min()
print(f"bandwidth: {bandwidth:.3f} eV, band center: {all_eigs.mean():.3f} eV")

wannier hamiltonian: 6 orbitals, 343 R-vectors
bandwidth: 19.514 eV, band center: 5.734 eV


## 9. DMFT with TRIQS CTHYB

single-site DMFT using the Wannier Hamiltonian:
- G_loc(iw) = (1/Nk) sum_k [iw + mu - H(k) - Sigma(iw)]^(-1)
- G0^(-1) = G_loc^(-1) + Sigma
- solve impurity problem for new Sigma
- double-counting: FLL formula

In [12]:
from triqs_cthyb import Solver
from triqs.operators import n
from triqs.gf import BlockGf, inverse
from tqdm.notebook import tqdm
import time

# dmft is off by default since niel doesn't need it
# turn on if you need spectral functions, correlated dos, etc.
RUN_DMFT = False

# qmc parameters - adjust based on your system and available time
# n_cycles: 5000 for quick tests, 50000+ for production
# length_cycle: 50 for small systems, 100+ for larger
# dmft_iterations: 5 for convergence check, 10-20 for production
QMC_N_CYCLES = 5000
QMC_LENGTH_CYCLE = 50
DMFT_ITERATIONS_TO_RUN = 5

# correlated subspace from d/f elements only
n_corr_orbitals = sum(orbital_info[e]['count'] * orbital_info[e]['atoms'] 
                      for e in correlated_elements) if correlated_elements else num_wann
n_corr = min(n_corr_orbitals, num_wann)

# inverse temperature: higher beta for narrower bands
TRIQS_BETA = max(20.0, 400.0 / bandwidth) if bandwidth > 0 else 40.0

# dmft k-mesh: denser for smaller cells
nk_dmft = max(4, min(12, int(20 / min(ase_atoms.cell.lengths()))))
kpts = [[i/nk_dmft, j/nk_dmft, k/nk_dmft] 
        for i in range(nk_dmft) for j in range(nk_dmft) for k in range(nk_dmft)]
Hk_all = np.array([get_Hk(k)[:n_corr, :n_corr] for k in kpts])

# matsubara frequencies
n_iw = max(256, min(512, int(TRIQS_BETA * bandwidth / 2)))

mu = all_eigs.mean()
gf_struct = [('up', n_corr), ('down', n_corr)]
S = Solver(beta=TRIQS_BETA, gf_struct=gf_struct, n_iw=n_iw)

# density-density interaction
H_int = sum(HUBBARD_U * n('up', i) * n('down', i) for i in range(n_corr))

Sigma = BlockGf(mesh=S.G_iw.mesh, gf_struct=gf_struct)
for bl in Sigma.indices:
    Sigma[bl].zero()
dc_shift = 0.0

# vectorized local green's function
def compute_G_loc(mu, Sigma, dc, pbar_desc="G_loc", show_pbar=True):
    G_loc = BlockGf(mesh=S.G_iw.mesh, gf_struct=gf_struct)
    for bl in G_loc.indices:
        G_loc[bl].zero()
    
    iw_vals = np.array([complex(w) for w in S.G_iw.mesh])
    n_iw_pts = len(iw_vals)
    eye = np.eye(n_corr)
    
    for spin in ['up', 'down']:
        sigma_data = Sigma[spin].data
        g_loc_data = np.zeros_like(sigma_data)
        
        iterator = range(n_iw_pts)
        if show_pbar:
            iterator = tqdm(iterator, desc=f"{pbar_desc} [{spin}]", leave=False, ncols=80)
        
        for iw_idx in iterator:
            iw = iw_vals[iw_idx]
            Gk_inv_all = (iw + mu - dc) * eye - Hk_all - sigma_data[iw_idx]
            g_loc_data[iw_idx] = np.linalg.inv(Gk_inv_all).sum(axis=0) / len(Hk_all)
        
        G_loc[spin].data[:] = g_loc_data
    return G_loc

print(f"dmft setup:")
print(f"  run dmft: {RUN_DMFT}")
print(f"  correlated orbitals: {n_corr} (from {correlated_elements or 'all'})")
print(f"  k-points: {nk_dmft}^3 = {len(Hk_all)}")
print(f"  beta: {TRIQS_BETA:.1f} (bandwidth={bandwidth:.2f} eV)")
print(f"  U: {HUBBARD_U} eV")

if not RUN_DMFT:
    print("\nskipping dmft (RUN_DMFT=False)")
    n_tot = 0.0
else:
    qmc_params = {
        'h_int': H_int,
        'n_cycles': QMC_N_CYCLES,
        'length_cycle': QMC_LENGTH_CYCLE,
        'n_warmup_cycles': min(2000, QMC_N_CYCLES // 4),
        'imag_threshold': 0.01
    }
    
    print(f"  qmc cycles: {QMC_N_CYCLES}")
    print(f"  dmft iterations: {DMFT_ITERATIONS_TO_RUN}")
    
    start = time.time()
    
    dmft_pbar = tqdm(range(DMFT_ITERATIONS_TO_RUN), desc="DMFT", ncols=100, 
                     bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]')

    for it in dmft_pbar:
        dmft_pbar.set_postfix_str(f"computing G_loc...")
        G_loc = compute_G_loc(mu, Sigma, dc_shift, pbar_desc=f"iter {it+1}")
        
        # dyson equation for bath green's function
        for spin in ['up', 'down']:
            S.G0_iw[spin] << inverse(inverse(G_loc[spin]) + Sigma[spin])
        
        dmft_pbar.set_postfix_str(f"QMC solve ({QMC_N_CYCLES} cycles)...")
        S.solve(**qmc_params)
        
        # extract self-energy from impurity solution
        for spin in ['up', 'down']:
            Sigma[spin] << inverse(S.G0_iw[spin]) - inverse(S.G_iw[spin])
        
        # fll double-counting correction
        n_tot = sum(S.G_iw[sp].density().real.trace() for sp in ['up', 'down'])
        dc_shift = HUBBARD_U * (n_tot / 2 - 0.5)
        
        dmft_pbar.set_postfix_str(f"n={n_tot:.4f}, dc={dc_shift:.3f}")

    print(f"\ncompleted in {time.time() - start:.1f}s, n={n_tot:.4f}")

dmft setup:
  run dmft: False
  correlated orbitals: 6 (from all)
  k-points: 4^3 = 64
  beta: 20.5 (bandwidth=19.51 eV)
  U: 0.0 eV

skipping dmft (RUN_DMFT=False)


## 10. Generate OMERE NIEL Input

compute non-ionizing energy loss (NIEL) for radiation damage calculations:
- uses lindhard partition function for nuclear/electronic stopping
- displacement threshold from DFT cohesive energy
- outputs in OMERE-compatible format

In [13]:
from ase.data import atomic_masses, atomic_numbers

avg_mass = np.mean(ase_atoms.get_masses())
avg_Z = np.mean([atomic_numbers[s] for s in ase_atoms.get_chemical_symbols()])
density = sum(ase_atoms.get_masses()) / ase_atoms.get_volume() * 1.66054  # g/cm³

# displacement threshold from bandwidth (semiconductors ~15-25, metals ~20-40 eV)
E_d = max(15.0, min(40.0, bandwidth * 2))

# lindhard partition: fraction of recoil energy going to atomic displacements
def lindhard_partition(T, A, Z):
    epsilon = 11.5 * T / (Z**(7/3))
    k = 0.1337 * Z**(1/6) * (A)**0.5
    g = epsilon + 0.40244 * epsilon**(3/4) + 3.4008 * epsilon**(1/6)
    return 1 / (1 + k * g)

# niel calculation using screened rutherford cross-section
def calculate_niel(E_MeV, A, Z, E_d):
    m_p = 938.3  # proton mass in MeV/c²
    E_max = 4 * E_MeV * m_p * A * 931.5 / (m_p + A * 931.5)**2
    E_min = E_d * 1e-6
    
    if E_max <= E_min:
        return 0.0
    
    a0 = 0.529e-8  # bohr radius in cm
    a_screen = 0.8853 * a0 / (Z**(1/3))  # thomas-fermi screening
    
    n_points = 50
    T_vals = np.logspace(np.log10(E_min), np.log10(E_max), n_points)
    
    niel = 0.0
    for i in range(len(T_vals) - 1):
        T = (T_vals[i] + T_vals[i+1]) / 2
        dT = T_vals[i+1] - T_vals[i]
        
        L = lindhard_partition(T * 1e6, A, Z)
        damage = T * L
        d_sigma = (Z * 1.44e-13)**2 / (4 * E_MeV * T**2) * (a_screen * 1e8)**2
        niel += damage * d_sigma * dT
    
    return niel * 6.022e23 / A  # convert to MeV·cm²/g

energies_MeV = np.arange(10, 105, 5)
niel_values = [calculate_niel(E, avg_mass, avg_Z, E_d) for E in energies_MeV]

omere_output_dir = os.path.join(os.path.dirname(os.getcwd()), "omere_inputs")
os.makedirs(omere_output_dir, exist_ok=True)
omere_file = os.path.join(omere_output_dir, f"{formula}_NIEL.dat")

with open(omere_file, 'w') as f:
    f.write("# Energy_MeV\tNIEL_MeV_cm2_g\n")
    for E, niel in zip(energies_MeV, niel_values):
        f.write(f"{E:.4e}\t{niel:.4e}\n\n")

print(f"material: {formula}")
print(f"avg mass: {avg_mass:.2f} amu, avg Z: {avg_Z:.1f}")
print(f"density: {density:.3f} g/cm³")
print(f"displacement threshold: {E_d:.1f} eV (from bandwidth)")
print(f"\nniel (MeV·cm²/g):")
for E, niel in list(zip(energies_MeV, niel_values))[:5]:
    print(f"  {E:3.0f} MeV: {niel:.4e}")
print(f"  ...")
print(f"\nsaved: {omere_file}")

material: InP
avg mass: 72.90 amu, avg Z: 32.0
density: 4.585 g/cm³
displacement threshold: 39.0 eV (from bandwidth)

niel (MeV·cm²/g):
   10 MeV: 3.9217e-05
   15 MeV: 2.6138e-05
   20 MeV: 1.9599e-05
   25 MeV: 1.5676e-05
   30 MeV: 1.3061e-05
  ...

saved: /home/vm/LUMENS-PV/omere_inputs/InP_NIEL.dat
